## A simple model for $R \equiv ^{14}\!\!C/^{12}C$ normalized so that $R=1$ in the atmosphere

$$ \frac{\partial R}{\partial t} + [\mathbf{TRdiv}]R = \kappa\boldsymbol{\Lambda}(1-R) -\lambda R $$

Here we will redo the radiocarbon age example, but using the OCIM transport operator made available through the AIBECS package

In [1]:
using AIBECS
const mask, grd, T_OCIM = OCIM1.load() ;

┌ Warning: Error requiring Unitful from Flatten:
│ ArgumentError: Package Flatten does not have Unitful in its dependencies:
│ - If you have Flatten checked out for development and have
│   added Unitful as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Flatten
│ Stacktrace:
│  [1] require(::Module, ::Symbol) at ./loading.jl:836
│  [2] top-level scope at /Users/fprimeau/.julia/packages/Flatten/AXBiq/src/Flatten.jl:21
│  [3] eval at ./boot.jl:328 [inlined]
│  [4] eval at /Users/fprimeau/.julia/packages/Flatten/AXBiq/src/Flatten.jl:1 [inlined]
│  [5] (::getfield(Flatten, Symbol("##3#6")))() at /Users/fprimeau/.julia/packages/Requires/9Jse8/src/require.jl:67
│  [6] err(::getfield(Flatten, Symbol("##3#6")), ::Module, ::String) at /Users/fprimeau/.julia/packages/Requires/9Jse8/src/require.jl:38
│  [7] #2 at /Users/fprimeau/.julia/packages/Requires/9Jse8/src/require.jl:66 [inlined]
│  [8] 

Loading OCIM1 with JLD2 ✅


Some useful OCIM stuff

In [2]:
iwet = findall(x -> x == 1, vec(mask));  # index to wet gridboxes
dz1 = grd["dzt"][1]                      # thickness of the top layer
z = vec(grd["ZT3d"])[iwet]               # depth of the gridbox centers
nwet = length(iwet)                      # number of wet grid boxes
dv = vec(grd["DZT3d"])[iwet].*vec(grd["DYT3d"])[iwet].*vec(grd["DXT3d"])[iwet]; # volume of the gridboxes
lat, lon = vec(grd["yt"]), vec(grd["xt"]); # latitudes and longitudes (useful for plotting)
depth = vec(grd["zt"]) ;

Make a table of parameters

In [3]:
t = empty_parameter_table()               # initialize table of parameters
add_parameter!(t, :λ, 1 / (5730*log(2))u"yr") # add the radioactive decay e-folding timescale
add_parameter!(t, :κ, 50u"m" / 10u"yr")
initialize_Parameters_type(t)             # Generate the parameter table

The model parameters

In [4]:
p₀ = Parameters()

     λ = 2.52e-04 [yr⁻¹] (fixed)
     κ = 5.00e+00 [m yr⁻¹] (fixed)


Parameters{Float64}


The sources and sinks of $R$

In [5]:
function sms_14c(R, p)
    λ = p.λ
    κ = p.κ
    return κ * (z .< 20) .* (1.0 .- R) / dz1 - λ.*R
end

sms_14c (generic function with 1 method)

The tracer transport operator

In [6]:
T_14c(p) = T_OCIM

T_14c (generic function with 1 method)

Prepare the stuff needed for the AIBECS solver:

In [7]:
T_matrices = (T_14c,)            # bundles all the transport matrices in a tuple
sources_minus_sinks = (sms_14c,) # bundles all the source-sink functions in a tuple
F, ∇ₓF = state_function_and_Jacobian(T_matrices, sources_minus_sinks, nwet) # generates the state function (and its Jacobian!)

(getfield(AIBECS, Symbol("#F#22")){getfield(AIBECS, Symbol("#T#18")){Tuple{typeof(T_14c)}},getfield(AIBECS, Symbol("#G#20")){Tuple{typeof(sms_14c)},getfield(AIBECS, Symbol("#tracers#17")){Int64,Int64}}}(getfield(AIBECS, Symbol("#T#18")){Tuple{typeof(T_14c)}}((T_14c,)), getfield(AIBECS, Symbol("#G#20")){Tuple{typeof(sms_14c)},getfield(AIBECS, Symbol("#tracers#17")){Int64,Int64}}((sms_14c,), getfield(AIBECS, Symbol("#tracers#17")){Int64,Int64}(200160, 1))), getfield(AIBECS, Symbol("#∇ₓF#24")){getfield(AIBECS, Symbol("#T#18")){Tuple{typeof(T_14c)}},getfield(AIBECS, Symbol("#∇ₓG#23")){Tuple{typeof(sms_14c)},Int64,Int64}}(getfield(AIBECS, Symbol("#T#18")){Tuple{typeof(T_14c)}}((T_14c,)), getfield(AIBECS, Symbol("#∇ₓG#23")){Tuple{typeof(sms_14c)},Int64,Int64}((sms_14c,), 200160, 1)))

In [8]:
x₀ = ones(nwet)                           # initial iterate for the solver
prob = SteadyStateProblem(F, ∇ₓF, x₀, p₀) # define the problem
R = solve(prob, CTKAlg())                 # solve the problem
c14age = -log.(R)/p₀.λ;                   # convert R to $^{14}C-age$

Make some plots

In [9]:
c14age_3d = NaN * mask     # creates a 3D array of NaNs
c14age_3d[iwet] = c14age   # Fills the wet grid boxes with the age values
size(c14age_3d)            # Just to check the size of age_3D

(91, 180, 24)

Pick a layer to plot

In [10]:
iz = findfirst(depth .> 700) # aim for a depth of ~ 700 m
iz, depth[iz]
c14age_3d_1000m_yr = c14age_3d[:,:,iz] * ustrip(1.0u"s" |> u"yr")

91×180 Array{Float64,2}:
  NaN       NaN       NaN       NaN      …   NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN      …   NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
  NaN       NaN       NaN       NaN          NaN       NaN       NaN    
 1342.82    NaN       NaN       NaN          NaN       NaN      1342.79 
 1339.91   1336.99   1334.25   1329.63   …  1342.4    1342.14   1341.25 
 1346.55   1343.69   1340.57   1336.83      1348.46   1348.21   1347.44 
 1350.4    1347.7    1345.53   1343.22      1353.99   1352.69   1351.33 
    ⋮                     

In [11]:
ENV["MPLBACKEND"]="qt5agg"
using PyPlot, PyCall

In [ ]:
clf()
ccrs = pyimport("cartopy.crs")
ax = subplot(projection=ccrs.Robinson(central_longitude=-155.0))
ax.coastlines()
lon_cyc = [lon; 360+lon[1]] # making it cyclic for Cartopy
age_cyc = hcat(c14age_3d_1000m_yr, c14age_3d_1000m_yr[:,1])
p = contourf(lon_cyc, lat, age_cyc, levels=0:100:1800, transform=ccrs.PlateCarree(), zorder=-1)
colorbar(p, orientation="horizontal")
gcf() # gets the current figure to display

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*